### Example (Gram-Schmidt via manual QR) 
source: Professor Park's notes $\\$
Find the **reduced** QR factorization of the following matrix on computers. Use the result to five the answers to a mathematical question: Give an orthogonal system (not necessarily of unit length) of vectors that spans the same space as $\text{span}{(1,2,2),(-4,3,2)}$

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def qr_red_GS(A):
    '''
    Returns the reduced QR factorization of a matrix 
    using the Gram-Schmidt process

    Input: 
    A - array, matrix to be factored
    Outputs:
    Q - array, orthogonal matrix
    R - array, upper triangular matrix
    '''
    # get m, n via the dimensions of A
    (m, n) = A.shape
    if m < n: # need A to be a tall matrix in context of least squares
        raise ValueError("m must be greater than n")
    
    Q = np.zeros((m,n))
    R = np.zeros((n,n))

    for j in range(n):
        y = A[:, j].copy()
        for i in range(j):
            R[i, j] = np.dot(Q[:, i], y)
            y = y - R[i, j] * Q[:, i]
        R[j, j] = np.linalg.norm(y)
        Q[:, j] = y / R[j, j]

    return Q, R

In [5]:

"""Toggle comment to test different matrices"""
A = np.array([[1,2,2],[-4,3,2]], dtype=np.float64)
# A = np.array([[1,2,2, -1],[-4,3,2, 3], [1, 1, 1, 1]], dtype=np.float64)
# A = np.array([[1,2,2, -1, 4],[-4,3,2, 3, -2], [1, 1, -1, 1,0]], dtype=np.float64)

A = A.T

Q, R = qr_red_GS(A)

print("A = QR? --->", np.allclose(A, Q @ R))
print("A\n", A)
print("Q\n", Q)
print("R\n", R)
print("QR\n", Q @ R)

A = QR? ---> True
A
 [[ 1. -4.]
 [ 2.  3.]
 [ 2.  2.]]
Q
 [[ 0.33333333 -0.93333333]
 [ 0.66666667  0.33333333]
 [ 0.66666667  0.13333333]]
R
 [[3. 2.]
 [0. 5.]]
QR
 [[ 1. -4.]
 [ 2.  3.]
 [ 2.  2.]]


In [6]:
"""
Inner loop must be `for i in range(j)` instead of `for i in range(j-1)`.

Since j starts from 0, not 1, we shouldn't use j-1 (this leads to j-2 in effect)
e.g., if j = 2 (3rd iteration) --> for i in range(j) <==> i in [0, 1] 
  (two inner iterations; correct b/c we are subtracting q1, q2 components) 
When `for i in range(j-1)` is used instead of `for i in range(j)`, 
it is missing one last iteration, and gives wrong result.
In particular, Q is not orthogonal anymore, i.e., Q^T Q != I.
"""
print("Q^T Q\n", Q.T @ Q)

Q^T Q
 [[1.00000000e+00 2.28212511e-17]
 [2.28212511e-17 1.00000000e+00]]


### Example (Gram-Schmidt via manual QR revisited)

Find the **full** QR factorization of the following matrix, whose columns are $(1,2,2),(-4,3,2)$

1. Pad A with columns of the identity matrix

In [9]:
def pad_matrix_eye(A):
    '''
    Pad the input matrix with columns of the identity matrix
    Input:
    A - array, matrix to be padded 
    '''
    (m, n) = A.shape
    if m < n:
        TRANSPOSE = True
        A = A.T
        (m, n) = A.shape
    else:
        TRANSPOSE = False
    
    # k = m - n
    #A_ = np.zeros((m, m))
    A_ = np.eye(A.shape[0])
    A_[:, :n] = A
    # append A with standard basis vectors

    if TRANSPOSE == True:
        A = A_.T
    
    return A_

In [10]:
# pad_matrix_eye sanity check

A = np.array([[1,2,2, -1],
              [-4,3,2, 3],
              #[1, 1, 1, 1]
              ], dtype=np.float64)
# A = A.T

print(pad_matrix_eye(A))

[[ 1. -4.  0.  0.]
 [ 2.  3.  0.  0.]
 [ 2.  2.  1.  0.]
 [-1.  3.  0.  1.]]


2. Full QR factorization

In [11]:
def qr_GS(A):
    """
    Return full QR factroization of a matrix using 
    Gram-Schmidt orthogonalization.

    Input:
        A (array): matrix to be factored (row-major assumed)
    Output:
        Q (array): orthogonal matrix
        R (array): upper triangular matrix
    Note:
        Input matrix is padded with columns of identity matrix.
    """
    (m, n) = A.shape
    if m < n:
        raise ValueError("The number of rows must be greater than or equal to the number of columns")
    
    A_ = pad_matrix_eye(A)
    R = np.zeros((m, n))

    Q, R_ = qr_red_GS(A_)

    # Construct matrix R: 
    #   R_ is computed to be m x m, but R take only n x n part (m >= n)
    R[:n, :] = R_[:n, :n]

    return Q, R

In [12]:
# A = np.array([[1,2,2],[-4,3,2]], dtype=np.float64)
A = np.array([[1,2,2, -1],[-4,3,2, 3], [1, 1, 1, 1]], dtype=np.float64)
# A = np.array([[1,2,2, -1, 4],[-4,3,2, 3, -2], [1, 1, -1, 1,0]], dtype=np.float64)
# d = 1e-10
# A = np.array([[1,d,0,0],[-4,3,2, 3], [1, 1, 1, 1]], dtype=np.float64)

A = A.T

Q, R = qr_GS(A)

print("A = QR? --->", np.allclose(A, Q @ R))
print("Q^T Q = I? --->", np.allclose(Q.T @ Q, np.eye(A.shape[0])))
print("A\n", A)
print("Q\n", Q)
print("R\n", R)
print("QR\n", Q @ R)
print("Q^T Q\n", Q.T @ Q)

A = QR? ---> True
Q^T Q = I? ---> True
A
 [[ 1. -4.  1.]
 [ 2.  3.  1.]
 [ 2.  2.  1.]
 [-1.  3.  1.]]
Q
 [[ 0.31622777 -0.70596229  0.6249268  -0.10527936]
 [ 0.63245553  0.39402546  0.01275361 -0.66676929]
 [ 0.63245553  0.22984819  0.06376804  0.73695553]
 [-0.31622777  0.54178501  0.7779701   0.03509312]]
R
 [[3.16227766 0.9486833  1.26491106]
 [0.         6.09097693 0.45969637]
 [0.         0.         1.47941855]
 [0.         0.         0.        ]]
QR
 [[ 1. -4.  1.]
 [ 2.  3.  1.]
 [ 2.  2.  1.]
 [-1.  3.  1.]]
Q^T Q
 [[ 1.00000000e+00 -3.66535855e-17 -4.64608460e-17  9.64140833e-16]
 [-3.66535855e-17  1.00000000e+00  2.87738379e-17  1.58036019e-15]
 [-4.64608460e-17  2.87738379e-17  1.00000000e+00  2.44866380e-16]
 [ 9.64140833e-16  1.58036019e-15  2.44866380e-16  1.00000000e+00]]
